In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-07-16"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
6378,2024-07-16,Argentina Liga A,22:10,Instituto de Cordoba,Boca Juniors,1.71,2.09,149.5,1.80,1.87,-3.5,2.01,2.40,ru2gnu7l,0.584795,0.478469,0.555556,0.534759,0.063264,0.0,0.6,NaN,NaN,143.556,24.168490,0.168356,123.982,26.592843,0.214490,89.428,133.506,87.32,123.97,0.0,0.0,1.0,0.0,0.141421,0.026974,0.125067,-0.18,-0.036,-19.722222,0.792999,0.8,0.007001,-1.91,-0.382,-2.853403,0.000000,0.0,0.000000
6379,2024-07-16,Eua Wnba,12:30,Washington Mystics F,Phoenix Mercury F,2.42,1.61,168.5,1.91,1.91,2.5,2.00,1.40,fRJsiSEd,0.413223,0.621118,0.523560,0.523560,0.034341,0.0,0.0,NaN,NaN,227.636,136.452168,0.599431,155.092,22.466601,0.144860,0.000,0.000,468.65,148.78,1.0,0.0,0.0,0.0,0.284246,0.000000,0.249567,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6380,2024-07-16,Eua Wnba,20:00,New York Liberty F,Connecticut Sun F,1.45,2.88,153.5,1.91,1.91,-4.5,1.91,4.40,MXBTEV63,0.689655,0.347222,0.523560,0.523560,0.036877,0.0,0.6,NaN,NaN,NaN,NaN,NaN,126.694,21.757782,0.171735,0.000,124.626,128.16,151.89,0.0,0.0,0.0,0.0,0.467050,0.000000,0.558065,0.00,0.000,inf,0.000000,0.0,0.000000,2.54,0.508,3.700787,0.000000,0.0,0.000000
6381,2024-07-16,República Dominicana Lnb,21:00,Leones,Marineros,1.91,1.81,160.5,1.85,1.85,3.5,1.55,1.48,8vPyrIcd,0.523560,0.552486,0.540541,0.540541,0.076046,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.038016,0.000000,0.032672,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6382,2024-07-16,Eua Nba,17:30,Brooklyn Nets,New York Knicks,1.69,2.15,178.5,1.85,1.89,-3.5,1.99,2.52,bgXPRGig,0.591716,0.465116,0.540541,0.529101,0.056832,0.2,0.6,NaN,NaN,233.240,145.817439,0.625182,267.606,66.163944,0.247244,235.922,274.690,128.26,295.61,0.0,0.0,0.0,0.0,0.169411,0.015125,0.166194,1.56,0.312,2.211538,0.540191,0.6,0.059809,1.48,0.296,3.885135,0.447052,0.6,0.152948
6383,2024-07-16,Eua Nba,19:30,Detroit Pistons,Chicago Bulls,2.16,1.68,173.5,1.82,1.92,1.5,1.99,1.52,SYOtOYjI,0.462963,0.595238,0.549451,0.520833,0.058201,0.6,0.8,NaN,NaN,287.436,181.894966,0.632819,218.438,41.683905,0.190827,292.414,272.372,177.12,187.46,0.0,0.0,0.0,0.0,0.176777,0.037813,0.189368,-1.06,-0.212,-5.471698,0.376824,0.4,0.023176,2.58,0.516,1.317829,0.443932,0.5,0.056068
6384,2024-07-16,Eua Nba,21:00,Minnesota Timberwolves,Philadelphia 76ers,1.51,2.53,175.5,1.80,1.92,-4.5,1.91,2.71,zaSlMCLU,0.662252,0.395257,0.555556,0.520833,0.057509,0.8,0.8,NaN,NaN,189.546,35.135354,0.185366,205.112,50.206029,0.244774,168.868,276.376,155.53,269.92,0.0,0.0,0.0,0.0,0.357054,0.045620,0.244885,-3.22,-0.644,-0.791925,0.583546,0.4,-0.183546,3.07,0.614,2.491857,0.491943,0.4,-0.091943
6385,2024-07-16,Eua Nba,21:30,Portland Trail Blazers,Washington Wizards,1.88,1.89,181.5,1.83,1.89,-1.5,1.97,2.01,ttucQOXo,0.531915,0.529101,0.546448,0.529101,0.061015,1.0,0.4,NaN,NaN,259.338,51.556503,0.198800,272.110,156.691593,0.575839,338.770,648.674,285.69,537.14,0.0,1.0,0.0,0.0,0.003751,0.022810,0.014213,-2.80,-0.560,-1.571429,0.340078,0.4,0.059922,0.02,0.004,222.500000,0.273440,0.4,0.126560
6386,2024-07-16,Eua Nba,23:30,San Antonio Spurs,New Orleans Pelicans,1.99,1.79,168.5,1.82,1.90,1.5,1.87,1.67,OOWDMtXA,0.502513,0.558659,0.549451,0.526316,0.061172,0.2,0.4,NaN,NaN,298.472,216.947498,0.726860,218.094,39.102338,0.179291,314.350,178.556,229.36,274.05,0.0,0.0,0.0,0.0,0.074826,0.030413,0.079899,-3.12,-0.624,-1.586538,0.382360,0.3,-0.082360,1.23,0.246,3.2

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,22:10,Argentina Liga A,Instituto de Cordoba,Boca Juniors,149.5,1.80,1.0000,Over
1,20:00,Eua Wnba,New York Liberty F,Connecticut Sun F,153.5,1.91,1.0000,Over
2,17:30,Eua Nba,Brooklyn Nets,New York Knicks,178.5,1.85,1.0000,Over
3,21:00,Eua Nba,Minnesota Timberwolves,Philadelphia 76ers,175.5,1.80,1.0000,Over
4,21:30,Eua Nba,Portland Trail Blazers,Washington Wizards,181.5,1.83,0.9989,Over
